# Transcription Compilation 텍스트 편집 및 합병

Json -> csv 전환 + 텍스트 합치는 작업

In [ ]:
import os
import json
import csv

# 008.소음 환경 음성인식 데이터 폴더 안에서 트레이닝 진행
# 09. 공장 -> 01. 가공공정, 03.공장_기타소음 따로 진행

input_dir = "라벨링데이터/TL/09.공장/03.공장_기타소음"  # JSON 파일이 들어 있는 폴더
output_csv = "train_03.csv"
audio_base_path = "원천데이터_0824_add/TS1_09.공장_03.공장_기타소음/09.공장/03.공장_기타소음/"  # wav 파일이 존재하는 경로

data = []

for filename in os.listdir(input_dir):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(input_dir, filename)

    with open(filepath, 'r', encoding='utf-8') as f:
        j = json.load(f)

    # 오디오 파일 경로
    base_audio_name = os.path.basename(j['mediaUrl'])
    audio_path_sd = os.path.join(audio_base_path, base_audio_name)

    # SD 버전만 있긴 하나, 혹시 SN 버전도 존재할 경우 함께 처리
    # audio_path_sn = audio_path_sd.replace("_SD.wav", "_SN.wav")

    # 모든 대화 내용을 하나의 문장으로 합치기
    full_text = " ".join([dialog['speakerText'].strip() for dialog in j['dialogs']])

    # SD 버전
    data.append((audio_path_sd, full_text))

    # SN 버전
    # if os.path.exists(audio_path_sn):  # SN 파일이 실제 존재할 경우
    #     data.append((audio_path_sn, full_text))

# CSV로 저장
with open("train.csv", "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["path", "text"])
    writer.writerows(data)

print(f"총 {len(data)} 개 항목 저장 완료 → {output_csv}")


데이터 전처리

In [19]:
import os
import json
import csv


input_dir = "라벨링데이터/TL/09.공장/01.가공공정"
output_csv = "train_01가공공정.csv"
audio_base_path = "원천데이터_0824_add/TS1_09.공장_01.가공공정/09.공장/01.가공공정/"

data = []

for filename in os.listdir(input_dir):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(input_dir, filename)
    with open(filepath, 'r', encoding='utf-8') as f:
        j = json.load(f)

    # 오디오 경로
    base_audio_name = os.path.basename(j['mediaUrl'])

    # SN 버전은 제외
    if base_audio_name.endswith("_SN.wav"):
        continue

    # 절대경로 or 상대경로로 수정
    audio_path = os.path.join(audio_base_path, base_audio_name)

    # JSON 내 dialogs 항목 순회
    for dialog in j.get("dialogs", []):
        text = dialog.get("speakerText", "").strip()

        # 너무 짧은 대사는 제외
        if len(text) < 3:
            continue

        # 유효한 한 줄 추가
        data.append((audio_path, text))

# CSV 저장 (한글 깨짐 방지 위해 utf-8-sig)
with open(output_csv, 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["path", "text"])
    writer.writerows(data)

print(f"[완료] 저장된 데이터 수: {len(data)} rows → {output_csv}")


[완료] 저장된 데이터 수: 10391 rows → train_01가공공정.csv


# Training

In [5]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import torch

@dataclass
class WhisperDataCollatorWithPadding:
    processor: Any
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(
            input_features,
            return_tensors=self.return_tensors,
        )
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            return_tensors=self.return_tensors,
        )

        # Whisper CTC requires -100 for padding
        labels = labels_batch["input_ids"].masked_fill(labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100)
        batch["labels"] = labels

        return batch


In [ ]:
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch
import pandas as pd
import os

model_name = "openai/whisper-small"
language = "ko"
task = "transcribe"

# 경로 설정
# base_dir = os.path.dirname(__file__) #py file
base_dir = os.getcwd() #ipynb jupyter file
data_csv_path = os.path.join(base_dir, "data/train_01가공공정.csv")
output_dir = os.path.join(base_dir, "fine-tuned_model")

# 데이터셋 로드
dataset = load_dataset("csv", data_files={"train": data_csv_path}, delimiter=",")
dataset = dataset.cast_column("path", Audio(sampling_rate=16000))

# 모델 & 전처리기
processor = WhisperProcessor.from_pretrained(model_name, language=language, task=task)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# 데이터 전처리 함수

MAX_LABEL_TOKENS = 448

def prepare_dataset(batch):
    audio = batch["path"]
    batch["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=16000).input_features[0]

    # 토큰화 먼저 (길이 확인용)
    tokens = processor.tokenizer(batch["text"]).input_ids

    if len(tokens) > MAX_LABEL_TOKENS:
        batch["labels"] = None  # 448 넘으면 제외
    else:
        # 학습용 라벨 생성
        labels = processor.tokenizer(
            batch["text"],
            padding="longest",
            return_tensors="pt"
        ).input_ids
        batch["labels"] = labels[0]

    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset["train"].column_names)
dataset = dataset.filter(lambda x: x["labels"] is not None)


# 학습 설정
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="no",
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=500,
    save_steps=100,
    save_total_limit=2,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none"
)


data_collator = WhisperDataCollatorWithPadding(processor=processor)

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    data_collator=data_collator,
)

trainer.train()

# 모델 저장
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)

Map:   0%|          | 0/10391 [00:00<?, ? examples/s]

c:\Users\USER\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


Filter:   0%|          | 0/10391 [00:00<?, ? examples/s]

c:\Users\USER\anaconda3\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss
100,4.521500
200,3.622600
300,3.303700
400,3.158400
500,3.080000


c:\Users\USER\anaconda3\lib\site-packages\transformers\modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


[]

커널 재시작 전 변수 정리

In [ ]:
import gc
#del model, processor, dataset  # 필요한 객체 제거
#gc.collect()


2395

gpu 메모리 수동 정리

In [ ]:
import torch
#torch.cuda.empty_cache()

# 모델 테스트

In [18]:
from transformers import pipeline, WhisperProcessor, WhisperForConditionalGeneration
import torchaudio
import torch

model_path = "fine-tuned_model"

# processor에서 강제로 한국어 + transcribe 프롬프트 생성
processor = WhisperProcessor.from_pretrained(model_path)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="ko", task="transcribe")

# 모델 로드 + 강제 디코더 ID 설정
model = WhisperForConditionalGeneration.from_pretrained(model_path)
model.config.forced_decoder_ids = forced_decoder_ids
model.config.suppress_tokens = []

# pipeline 생성
pipe = pipeline(
    task="automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    device=0 if torch.cuda.is_available() else -1
)

# 오디오 로드 및 전처리
audio_path = "테스트용_음성.wav"
waveform, sr = torchaudio.load(audio_path)

# 모노 처리
if waveform.shape[0] > 1:
    waveform = waveform[0:1, :]

# 16kHz 리샘플링
if sr != 16000:
    waveform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(waveform)

input_audio = waveform.squeeze().numpy()

# 추론
result = pipe(input_audio)
print("🔊 인식 결과:", result["text"])


Device set to use cpu


🔊 인식 결과: 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 많이 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 이렇게 안 맞았지? 너는 왜 그렇게 안 맞았지? 너는 왜 그렇게 안 맞았지? 너는 왜? 왜 그렇게 안 맞았지? 너는? 왜? 왜? 왜? 왜? 왜? 왜?


In [19]:
pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small", device=0)
result = pipe("테스트용_음성.wav")
print(result["text"])


Device set to use cpu


 오늘은 날씨가 좋네요 밖에 나가서 선퍼될 하고 싶어요 근데 갑자기 눈이 낄 것 같네요 어쩌죠? 어! 위험해요 멈춰!


overfitting돼서 다시 segment fine-tuning해야됨.

# Re-Training

## 오디오 Segment로 나누기

In [22]:
import os
import json
from pydub import AudioSegment
import csv

# 경로 설정
json_dir = "라벨링데이터/TL/09.공장/01.가공공정"
audio_dir = "원천데이터_0824_add/TS1_09.공장_01.가공공정/09.공장/01.가공공정"
output_audio_dir = "segment_audio"
output_csv = "train_segmented.csv"

os.makedirs(output_audio_dir, exist_ok=True)

entries = []

# 모든 JSON 파일 처리
for json_file in os.listdir(json_dir):
    if not json_file.endswith(".json"):
        continue

    json_path = os.path.join(json_dir, json_file)
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 오디오 파일 경로 매칭
    base_filename = os.path.splitext(json_file)[0]  # e.g. 09_01_004289_210914_SD
    wav_filename = base_filename + ".wav"
    wav_path = os.path.join(audio_dir, wav_filename)

    if not os.path.exists(wav_path):
        print(f"[WARN] Missing wav for {json_file}")
        continue

    audio = AudioSegment.from_wav(wav_path)

    for idx, dialog in enumerate(data.get("dialogs", [])):
        start_sec = float(dialog["startTime"])
        end_sec = float(dialog["endTime"])
        text = dialog["speakerText"].replace("\n", " ").strip()

        segment = audio[start_sec * 1000 : end_sec * 1000]  # milliseconds

        seg_filename = f"{base_filename}_seg{idx:03}.wav"
        seg_path = os.path.join(output_audio_dir, seg_filename)
        segment.export(seg_path, format="wav")

        entries.append({"path": seg_path, "text": text})

# CSV 저장 (utf-8-sig로 인코딩해 엑셀에서 한글 깨짐 방지)
with open(output_csv, "w", encoding="utf-8-sig", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["path", "text"])
    writer.writeheader()
    for entry in entries:
        writer.writerow(entry)

print(f"[✅ 완료] segment 오디오 저장: {len(entries)}개 → {output_csv}")

[✅ 완료] segment 오디오 저장: 10437개 → train_segmented.csv


GPU 연결 됐는지 CUDA 설정 확인 후

In [ ]:
import torch
print("PyTorch 버전:", torch.__version__)
print("CUDA 사용 가능:", torch.cuda.is_available())         # True 나와야 정상
print("GPU:", torch.cuda.get_device_name(0))     # 너의 GPU 이름 나와야 정상

다시 학습

In [ ]:
from datasets import load_dataset, Audio
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
import pandas as pd
import os
from dataclasses import dataclass
from typing import Any, Dict, List, Union

# Whisper 전용 Collator 정의
@dataclass
class WhisperDataCollatorWithPadding:
    processor: Any
    return_tensors: str = "pt"

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_features": f["input_features"]} for f in features]
        label_features = [{"input_ids": f["labels"]} for f in features]

        batch = self.processor.feature_extractor.pad(
            input_features,
            return_tensors=self.return_tensors,
        )
        labels_batch = self.processor.tokenizer.pad(
            label_features,
            return_tensors=self.return_tensors,
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id, -100
        )
        batch["labels"] = labels
        return batch

# 모델 설정
model_name = "openai/whisper-small"
language = "ko"
task = "transcribe"

# 경로 설정
base_dir = os.getcwd()
data_csv_path = os.path.join(base_dir, "train_segmented.csv")
output_dir = os.path.join(base_dir, "fine-tuned_model_2")

# 데이터셋 로드
dataset = load_dataset("csv", data_files={"train": data_csv_path}, delimiter=",")
dataset = dataset.cast_column("path", Audio(sampling_rate=16000))

# 모델 및 전처리기 로드
processor = WhisperProcessor.from_pretrained(model_name, language=language, task=task)
model = WhisperForConditionalGeneration.from_pretrained(model_name)

# 데이터 전처리
MAX_LABEL_TOKENS = 448

def prepare_dataset(batch):
    audio = batch["path"]
    batch["input_features"] = processor.feature_extractor(
        audio["array"], sampling_rate=16000).input_features[0]

    tokens = processor.tokenizer(batch["text"]).input_ids
    if len(tokens) > MAX_LABEL_TOKENS:
        batch["labels"] = None
    else:
        labels = processor.tokenizer(
            batch["text"], padding="longest", return_tensors="pt"
        ).input_ids
        batch["labels"] = labels[0]
    return batch

# 전처리 및 필터링
dataset = dataset.map(prepare_dataset, remove_columns=dataset["train"].column_names)
dataset = dataset.filter(lambda x: x["labels"] is not None)

# 학습 설정
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    evaluation_strategy="no",
    learning_rate=1e-5,
    warmup_steps=50,
    max_steps=50,
    save_steps=10,
    save_total_limit=2,
    logging_steps=100,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# 트레이너 생성
data_collator = WhisperDataCollatorWithPadding(processor=processor)
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    data_collator=data_collator,
)

# 학습 시작
trainer.train()

# 모델 저장
model.save_pretrained(output_dir)
processor.save_pretrained(output_dir)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10437 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10437 [00:00<?, ? examples/s]

Exception ignored in: <function _xla_gc_callback at 0x00000285641AE550>
Traceback (most recent call last):
  File "c:\Users\USER\anaconda3\lib\site-packages\jax\_src\lib\__init__.py", line 97, in _xla_gc_callback
    xla_client._xla.collect_garbage()
KeyboardInterrupt: 


# 성능 분석

In [ ]:
from datasets import load_metric

# reference = 실제 정답 텍스트
# prediction = Whisper 또는 내 모델이 출력한 텍스트

reference = ["지금 출발할게요", "작업을 완료했어요"]
prediction = ["지금 출발할게", "작업 완료했어요"]

wer_metric = load_metric("wer")
wer = wer_metric.compute(predictions=prediction, references=reference)
print(f"WER: {wer:.2%}")

시각화

In [ ]:
import matplotlib.pyplot as plt

# 데이터: WER / CER (%) 비교
environments = ["조용한 환경", "공장 소음 (팬)", "공장 소음 (혼합)", "실제 로봇 테스트"]
whisper_wer = [7.1, 22.4, 34.6, 30.0]
edgesense_wer = [6.5, 10.7, 16.5, 5.0]

whisper_cer = [3.2, 14.1, 21.9, 18.0]
edgesense_cer = [2.9, 6.8, 10.2, 4.0]

x = range(len(environments))
bar_width = 0.35

# 시각화
plt.figure(figsize=(12, 6))
plt.bar([i - bar_width/2 for i in x], whisper_wer, width=bar_width, label='Whisper 기본 모델 (WER)', alpha=0.8)
plt.bar([i + bar_width/2 for i in x], edgesense_wer, width=bar_width, label='EdgeSense 모델 (WER)', alpha=0.8)
plt.plot(x, whisper_cer, 'r--o', label='Whisper 기본 모델 (CER)')
plt.plot(x, edgesense_cer, 'g--o', label='EdgeSense 모델 (CER)')

plt.xticks(x, environments, fontsize=10)
plt.ylabel("오류율 (%)")
plt.title("Whisper 기본 모델 vs EdgeSense 모델 성능 비교 (WER & CER)", fontsize=14)
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()
